<a href="https://colab.research.google.com/github/nmq443/cognitive-science-final-project/blob/quang-branch/torcheeg_atcnet-with_preprocessed_data_cwt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# if run on colab
"""
!pip install torcheeg
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
"""

"\n!pip install torcheeg\nfrom google.colab import drive\ndrive.mount('/content/drive', force_remount=True)\n"

In [3]:
# if run on kaggle
!pip install torcheeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.5/214.5 kB 3.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.8/230.8 kB 14.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.2/299.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 71.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.3 MB/s eta 0:00:00
  Created wheel for torcheeg: f

In [3]:
import torcheeg
from torcheeg import transforms
from torcheeg.datasets import BCICIV2aDataset
from torcheeg.model_selection import KFoldGroupbyTrial
from torch.utils.data import DataLoader
from torcheeg.models import ATCNet, EEGNet
import torch

from torcheeg.trainers import ClassifierTrainer

import pytorch_lightning as pl

In [4]:
# if run on kaggle
# root_data_path = '/kaggle/input/bci-competition-iv-dataset-2a-in-mat-format/BCICIV-2a-mat'
# if run on colab
# root_data_path = '/content/drive/MyDrive/BCICIV-2a-mat'

# if run on local machine
root_data_path = './BCICIV-2a-mat/'

In [5]:
dataset = BCICIV2aDataset(
    root_path=root_data_path,
    io_path=f'./examples_pipeline/bciciv-2a',
    online_transform=transforms.Compose([
        transforms.To2d(),
        transforms.MinMaxNormalize(),
        transforms.ToTensor(),
        transforms.RandomNoise(p=0.3),
    ]),
    label_transform=transforms.Compose([
        transforms.Select('label'),
        transforms.Lambda(lambda x: x - 1)
    ]),
    chunk_size=7*250,
    num_worker=2
)

[2024-05-22 13:43:42] INFO (torcheeg/MainThread) 🔍 | Detected cached processing results, reading cache from ./examples_pipeline/bciciv-2a.


In [6]:
print("Dataset's info: ")
print(dataset.info)

Dataset's info: 
      start_at  end_at   clip_id subject_id  trial_id session subject  run  \
0          251    2001    A02E_0        A02         0       E     A02    3   
1         2254    4004    A02E_1        A02         1       E     A02    3   
2         4172    5922    A02E_2        A02         2       E     A02    3   
3         6124    7874    A02E_3        A02         3       E     A02    3   
4         8132    9882    A02E_4        A02         4       E     A02    3   
...        ...     ...       ...        ...       ...     ...     ...  ...   
5179     86751   88501  A07T_283        A07        43       T     A07    8   
5180     88657   90407  A07T_284        A07        44       T     A07    8   
5181     90585   92335  A07T_285        A07        45       T     A07    8   
5182     92699   94449  A07T_286        A07        46       T     A07    8   
5183     94758   96508  A07T_287        A07        47       T     A07    8   

      label  _record_id  
0         1   _recor

In [7]:
k_fold = KFoldGroupbyTrial(
    n_splits=10,
    split_path='./examples_pipeline/split',
    shuffle=True,
    random_state=44
)

In [8]:
DEVICE = 'gpu' if torch.cuda.is_available() else 'cpu'

for i, (train_dataset, val_dataset) in enumerate(k_fold.split(dataset)):
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=64,
        shuffle=True,
        num_workers=8
    )
    val_loader = DataLoader(
        dataset=val_dataset,
        batch_size=64,
        shuffle=False,
        num_workers=8
    )

    model = EEGNet(
        num_classes=4,
        num_electrodes=22,
        chunk_size=7*250,
    )

    trainer = ClassifierTrainer(
        model=model,
        num_classes=4,
        lr=1e-4,
        weight_decay=1e-4,
        accelerator=DEVICE
    )

    trainer.fit(
        train_loader,
        val_loader,
        max_epochs=50,
        default_root_dir=f'./examples_pipeline/eegnet_model/{i}',
        callbacks=[pl.callbacks.ModelCheckpoint(save_last=True)],
        enable_progress_bar=True,
        enable_model_summary=True,
        limit_val_batches=0.0
    )

    score = trainer.test(
        val_loader,
        enable_progress_bar=True,
        enable_model_summary=True
    )[0]
    print(f"Fold {i} test accuracy: {score['test_accuracy']: .4f}")

[2024-05-22 13:43:56] INFO (torcheeg/MainThread) 📊 | Detected existing split of train and test set, use existing split from ./examples_pipeline/split.
[2024-05-22 13:43:56] INFO (torcheeg/MainThread) 💡 | If the dataset is re-generated, you need to re-generate the split of the dataset instead of using the previous split.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: examples_pipeline/eegnet_model/0/lightning_logs
2024-05-22 13:43:56.817169: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
------

Training: |                                            | 0/? [00:00<?, ?it/s]

/home/quang/micromamba/envs/dl/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at /opt/conda/conda-bld/pytorch_1712608935911/work/aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
[2024-05-22 13:44:05] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.418 train_accuracy: 0.247 

[2024-05-22 13:44:13] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.405 train_accuracy: 0.253 

[2024-05-22 13:44:21] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.389 train_accuracy: 0.267 

[2024-05-22 13:44:28] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.395 train_accuracy: 0.273 

[2024-05-22 13:44:36] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.389 train_accuracy: 0.272 

[2024-05-22 13:44:43] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.383 tr

Testing: |                                             | 0/? [00:00<?, ?it/s]

[2024-05-22 13:50:13] INFO (torcheeg/MainThread) 
[Test] test_loss: 1.309 test_accuracy: 0.389 



...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: examples_pipeline/eegnet_model/4/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params
---------------------------------------------------
0 | model         | EEGNet           | 4.9 K 
1 | ce_fn         | CrossEntropyLoss | 0     
2 | train_loss    | MeanMetric       | 0     
3 | val_loss      | MeanMetric       | 0     
4 | test_loss     | MeanMetric       | 0     
5 | train_metrics | MetricCollection | 0     
6 | val_metrics   | MetricCollection | 0     
7 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
4.9 K     Trainable params
0         Non-trainable params
4.9 K     Total params
0.020     Total estimated model params size (MB)


Training: |                                            | 0/? [00:00<?, ?it/s]

[2024-05-22 14:10:10] INFO (torcheeg/MainThread) 
[Train] train_loss: 1.399 train_accuracy: 0.254 

/home/quang/micromamba/envs/dl/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Exception ignored in: <function _releaseLock at 0x762476e41990>
Traceback (most recent call last):
  File "/home/quang/micromamba/envs/dl/lib/python3.10/logging/__init__.py", line 228, in _releaseLock
    def _releaseLock():
KeyboardInterrupt: 
Exception ignored in: 

Testing: |                                             | 0/? [00:00<?, ?it/s]

<function _MultiProcessingDataLoaderIter.__del__ at 0x7623ab19eb00><object repr() failed>
Traceback (most recent call last):

KeyboardInterrupt

AssertionError: can only test a child process

KeyboardInterrupt



In [ ]:
!mkdir weights

In [ ]:
atc_weights_path = './weights/atc_weights.pt'
torch.save(model.state_dict(), atc_weights_path)
# to load weight:
# model.load_state_dict(torch.load(atc_weights_path))